In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import os
ROOT = '/content/drive/MyDrive/Midfluencer'
INPUT = os.path.join(ROOT, "images")
OUTPUT = os.path.join(ROOT, "output")

In [ ]:
MODEL = "stabilityai/stable-diffusion-2-1"

In [ ]:
!pip install -U autotrain-advanced

In [ ]:
!autotrain setup --update-torch

> INFO    Installing latest xformers
> INFO    Successfully installed latest xformers
> INFO    Installing latest PyTorch
> INFO    Successfully installed latest PyTorch


In [ ]:
!autotrain dreambooth \
--model stabilityai/stable-diffusion-xl-base-1.0 \
--image-path /content/img \
--prompt "photo of woman on the timesquare, photorealistic" \
--resolution 1024 \
--batch-size 1 \
--num-steps 500 \
--fp16 \
--gradient-accumulation 4 \
--lr 1e-4 \
--project-name midfluencer_v2 \
--use-8bit-adam

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.1.0+cu118)
    Python  3.10.13 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
> INFO    Namespace(version=False, model='stabilityai/stable-diffusion-xl-base-1.0', revision=None, tokenizer=None, image_path='/content/img', class_image_path=None, prompt='photo of woman on the timesquare, photorealistic', class_prompt=None, num_class_images=100, class_labels_conditioning=None, prior_preservation=None, prior_loss_weight=1.0, project_name='midfluencer_v2', seed=42, resolution=1024, center_crop=None, train_text_encoder=None, batch_size=1, sample_batch_size=4, epochs=1, num_steps=500, checkpointing_steps=100000, resume_from_checkpoint=None, gradient_accumulation=4, gradien

In [ ]:
from diffusers import DiffusionPipeline, StableDiffusionXLImg2ImgPipeline
import torch

version = "_v6"
if not os.path.exists(os.path.join(OUTPUT, f"raw{version}")):
  os.mkdir(os.path.join(OUTPUT, f"raw{version}"))
if not os.path.exists(os.path.join(OUTPUT, f"processed{version}")):
  os.mkdir(os.path.join(OUTPUT, f"processed{version}"))

pipe = DiffusionPipeline.from_pretrained(
    MODEL,
    torch_dtype=torch.float16
)
pipe.to("cuda")
pipe.load_lora_weights(os.path.join(OUTPUT, "midfluencer"+version), weight_name="pytorch_lora_weights.safetensors")

prompt = "create a photorealistic image of Laura on the timesquare"

for seed in range(10):
    generator = torch.Generator("cuda").manual_seed(seed)
    image_pipe = pipe(prompt=prompt, generator=generator, num_inference_steps=25)
    image = image_pipe.images[0]
    image.save(os.path.join(OUTPUT, f"raw{version}", f"{seed}.png"))

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.21.4",
  "_name_or_path": "stabilityai/stable-diffusion-xl-base-1.0",
  "feature_extractor": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "EulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [ ]:
pipe.to("cpu")
del pipe

In [ ]:
from PIL import Image
import os

refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    torch_dtype=torch.float16,
)
refiner.to("cuda")

all_images = os.listdir(os.path.join(OUTPUT, f"raw{version}"))
for image_file in all_images:
  image = Image.open(os.path.join(os.path.join(OUTPUT, f"raw{version}"), image_file))

  image = refiner(prompt="make the sourronding look more realistic and like a normal human in this scenery", generator=generator, image=image)
  image = image.images[0]
  image.save(os.path.join(OUTPUT, f"processed{version}", f"refined_{image_file}"))